## Imports

In [1]:
import os

import pyrootutils

root = pyrootutils.setup_root(
    search_from=os.path.dirname(os.getcwd()),
    indicator=[".git", "pyproject.toml"],
    pythonpath=True,
    dotenv=True,
)

if os.getenv("DATA_ROOT") is None:
    os.environ["DATA_ROOT"] = f"{root}"

In [2]:
from pathlib import Path
import cv2
import hydra
from hydra import compose, initialize
import supervision as sv

In [3]:
import torch

# Setup device-agnostic code
if torch.cuda.is_available():
    DEVICE = "cuda"  # NVIDIA GPU
    print("GPU Found!!")
else:
    raise Exception("No GPU Found!!")

GPU Found!!


## Paths setup

In [4]:
from omegaconf import DictConfig

with initialize(config_path="../configs", job_name="EDA", version_base=None):
    cfg: DictConfig = compose(config_name="train.yaml")
    # print(OmegaConf.to_yaml(cfg))
    print(cfg.paths)

{'root_dir': '${oc.env:PROJECT_ROOT}', 'root_data_dir': '${oc.env:DATA_ROOT}', 'results_dir': '${paths.root_dir}/results', 'log_dir': '${paths.root_dir}/logs/', 'output_dir': '${hydra:runtime.output_dir}', 'work_dir': '${hydra:runtime.cwd}', 'pretrained_model_dir': '${paths.root_dir}/pretrained_models'}


In [5]:
cfg.paths.pretrained_model_dir

'/workspaces/football-players-tracking-yolo/pretrained_models'

In [6]:
# auto reload libs
%load_ext autoreload
%autoreload 2

## Download Datasets

In [7]:
ROOT_DIR = Path(cfg.paths.root_dir)
ROOT_DATA_DIR = Path(cfg.paths.root_data_dir)
DATA_DIR = ROOT_DATA_DIR / cfg.datasets.datasets_dir
DATASET  = cfg.datasets.roboflow
DATA_DIR.mkdir(parents=True, exist_ok=True)
DATASET_DIR = DATA_DIR/ cfg.datasets.dataset_name
CLASS_NAMES = cfg.datasets.names
PRETRAINED_MODEEL_DIR = Path(cfg.paths.pretrained_model_dir)

In [8]:
if len(list(DATA_DIR.iterdir())) == 0:
    from roboflow import Roboflow

    rf = Roboflow()
    project = rf.workspace(DATASET.workspace).project(DATASET.project)
    version = project.version(DATASET.version)
    dataset = version.download(model_format=DATASET.model)

## Paths Setup

In [9]:
TRAIN_IMAGES_DIR = DATASET_DIR / "train" / "images"
TRAIN_LABELS_DIR = DATASET_DIR / "train"  / "labels"

VALID_IMAGES_DIR = DATASET_DIR / "valid" / "images"
VALID_LABELS_DIR = DATASET_DIR / "valid"  / "labels"

TEST_IMAGES_DIR = DATASET_DIR / "test" / "images"
TEST_LABELS_DIR = DATASET_DIR / "test"  / "labels"

## YOLO model import

In [ ]:
MODEL_NAME = f'{PRETRAINED_MODEEL_DIR}/{cfg.model.model_name.lower()}.pt'
MODEL_NAME

In [ ]:
from ultralytics import YOLO
model = YOLO(MODEL_NAME)

In [11]:
cfg.trainer.params.project

'/workspaces/football-players-tracking-yolo/results'

In [ ]:
results = model.train(data=cfg.datasets.dataset_yaml,
            **cfg.trainer.params,
            **cfg.datasets.augmentations,
)
            

Ultralytics 8.3.111 🚀 Python-3.11.11 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3080, 9898MiB)
engine/trainer: task=detect, mode=train, model=/workspaces/football-players-tracking-yolo/pretrained_models/yolo12l.pt, data=/workspaces/football-players-tracking-yolo/data/football-players-detection-14/data.yaml, epochs=3, time=None, patience=100, batch=0.8, imgsz=640, save=True, save_period=-1, cache=True, device=None, workers=16, project=/workspaces/football-players-tracking-yolo/results, name=augumented-data-yolo12l, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_n

E0000 00:00:1745064012.265919    2764 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745064012.268811    2764 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745064012.278404    2764 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745064012.278418    2764 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745064012.278419    2764 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745064012.278421    2764 computation_placer.cc:177] computation placer already registered. Please check linka

Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  2    173824  ultralytics.nn.modules.block.C3k2            [128, 256, 2, True, 0.25]     
  3                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  4                  -1  2    691712  ultralytics.nn.modules.block.C3k2            [256, 512, 2, True, 0.25]     
  5                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  6                  -1  4   4272944  ultralytics.nn.modules.block.A2C2f           [512, 512, 4, True, 4, True, 1.2]
  7                  -1  1   2360320  ultralyt

train: Scanning /workspaces/football-players-tracking-yolo/data/football-players-detection-14/train/labels.cache... 298 images, 0 backgrounds, 0 corrupt: 100%|██████████| 298/298 [00:00<?, ?it/s]

WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        2/3      6.55G       1.42     0.9398     0.8862        167        640: 100%|██████████| 50/50 [00:10<00:00,  4.75it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:00<00:00,  8.78it/s]

                   all         49       1174      0.641      0.468      0.461       0.27



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        3/3      6.53G      1.336     0.8656     0.8819        185        640:  96%|█████████▌| 48/50 [00:10<00:00,  4.76it/s]